In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
# !pip3 install pandas

In [6]:
import pyspark
import pyspark.sql
from pyspark.sql import *
import time
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
# from pyspark.streaming.kafka import KafkaUtils
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import pandas as pd

In [3]:
!pip3 install KafkaUtils

  Could not find a version that satisfies the requirement KafkaUtils (from versions: )
No matching distribution found for KafkaUtils


In [7]:
spark = SparkSession.builder.master("local").appName("api").getOrCreate()

In [10]:
df_gyro = spark.read.format("kafka").option("kafka.bootstrap.servers", "localhost:9092")\
        .option("subscribe", "gyro").load()
# df1 = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092")\
#         .option("subscribe", "gyro").load()
df_gyro_str = df_gyro.selectExpr("CAST(value AS STRING)")
df_gyro_struct = StructType()\
                    .add("Time", StringType())\
                    .add("TimeZone", StringType())\
                    .add("gy_x", DoubleType())\
                    .add("gy_y", DoubleType())\
                    .add("gy_z", DoubleType())
df_gyro_struct = df_gyro_str.select(from_json(col("value"), df_gyro_struct).alias("df_gyro"))
df_gyro_new = df_gyro_struct.select("df_gyro.*")
df_gyro = df_gyro_new.toPandas()

In [11]:
df_acc = spark.read.format("kafka").option("kafka.bootstrap.servers", "localhost:9092")\
        .option("subscribe", "acc").load()
df_acc_str = df_acc.selectExpr("CAST(value AS STRING)")
df_acc_struct = StructType()\
                    .add("Time", StringType())\
                    .add("TimeZone", StringType())\
                    .add("acc_x", DoubleType())\
                    .add("acc_y", DoubleType())\
                    .add("acc_z", DoubleType())
df_acc_struct = df_acc_str.select(from_json(col("value"), df_acc_struct).alias("df_acc"))
df_acc_new = df_acc_struct.select("df_acc.*")
df_acc = df_acc_new.toPandas()

In [12]:
df_acc['Time'] = list(map(lambda x:"".join(x.split(':')[:-1])+x.split(':')[-1][:2],list(df_acc['Time'])))
df_gyro['Time'] = list(map(lambda x:"".join(x.split(':')[:-1])+x.split(':')[-1][:2],list(df_gyro['Time'])))
df_acc.drop('TimeZone',axis=1,inplace=True)
df_gyro.drop('TimeZone',axis=1,inplace=True)
df_acc.drop_duplicates(['Time'],inplace=True)
df_gyro.drop_duplicates(['Time'],inplace=True)

In [13]:
# df_all = pd.concat([df_acc_new, df_gyro_new], sort = False)
df_all = pd.merge(df_acc,df_gyro,on='Time')
df_all

,Time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,2021-06-29 17063458,-0.229711,-0.031107,9.848858,0.35,-0.49,-0.42
1,2021-06-29 17063500,-0.193819,-0.081356,9.865608,0.07,-0.49,-0.42
2,2021-06-29 17063502,-0.165105,-0.105284,9.765109,0.14,-0.49,-0.42
3,2021-06-29 17063504,-0.191426,-0.122034,9.803394,0.21,-0.49,-0.42
4,2021-06-29 17063506,-0.205783,-0.069392,9.841680,0.28,-0.42,-0.42
...,...,...,...,...,...,...,...
6572,2021-06-29 17175460,-0.172283,-0.167498,9.791430,0.14,-0.56,-0.49
6573,2021-06-29 17175462,-0.174676,-0.186640,9.851252,0.07,-0.49,-0.42
6574,2021-06-29 17175464,-0.055035,-0.215354,9.801002,0.14,-0.49,-0.42
6575,2021-06-29 17175466,-0.126820,-0.203390,9.882358,0.21,-0.63,-0.42


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import expr
import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0,org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0,org.elasticsearch:elasticsearch-hadoop:7.6.2 pyspark-shell'

from pyspark import SparkContext,SparkConf
#    Spark Streaming
from pyspark.streaming import StreamingContext
from pyspark.sql.session import SparkSession
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame


# df2.writeStream.format("console").outputMode("complete").start()

In [9]:
df2 = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", "acc")\
    .load()
# .selectExpr("CAST(value AS STRING)")
#     .option("checkpointLocation","_")\

In [10]:
df2.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
df2.isStreaming

True

In [163]:
# df2 = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092")\
#         .option("subscribe", "acc").load()
# df2_str = df2.selectExpr("CAST(value AS STRING)")
# df2_struct = StructType()\
#                     .add("Time", StringType())\
#                     .add("TimeZone", StringType())\
#                     .add("acc_x", DoubleType())\
#                     .add("acc_y", DoubleType())\
#                     .add("acc_z", DoubleType())
# df2_struct = df2_str.select(from_json(col("value"), df2_struct).alias("df_acc"))
# df2_new = df2_struct.select("df_acc.*")
# df2_2 = df2_new.toPandas()

In [11]:
ds = df2 \
.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
   .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test") \
  .option("checkpointLocation","/home/test/meta")\
    .start()
# ds.awaitTermination()


# .trigger(processingTime='2 seconds')\

In [24]:
ds.isActive

False

In [37]:
ds2 = df2 \
    .selectExpr("CAST(value AS STRING)")\
   .writeStream \
    .format("console") \
    .start()
ds2.awaitTermination()

In [82]:
ds2.stop()

In [62]:
sq = df2 \
.writeStream.format('kafka').option("kafka.bootstrap.servers", "localhost:9092")\
    .option("subscribe", "test")\
    .option("checkpointLocation","_")\
    .start()
# sq.processAllAvailable() # Wait a bit to generate the runtime plans.
# sq.explain()

# sq.awaitTermination()

In [46]:
# sq.processAllAvailable() 

In [64]:
sq.explain()

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@5d19a3a5
+- *(1) Project [key#145, value#146, topic#147, partition#148, offset#149L, timestamp#150, timestampType#151]
   +- MicroBatchScan[key#145, value#146, topic#147, partition#148, offset#149L, timestamp#150, timestampType#151] class org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan




In [65]:
sq.isActive

False

In [61]:
sq.stop()

In [30]:
import warnings

from pyspark import copy_func, since, _NoValue
from pyspark.rdd import RDD, _load_from_socket, _local_iterator_from_socket, \
    ignore_unicode_prefix
from pyspark.serializers import BatchedSerializer, PickleSerializer, \
    UTF8Deserializer
from pyspark.storagelevel import StorageLevel
from pyspark.traceback_utils import SCCallSiteSync
from pyspark.sql.types import _parse_datatype_json_string
from pyspark.sql.column import Column, _to_seq, _to_list, _to_java_column
from pyspark.sql.readwriter import DataFrameWriter
from pyspark.sql.streaming import DataStreamWriter
from pyspark.sql.types import *
from pyspark.sql.pandas.conversion import PandasConversionMixin
from pyspark.sql.pandas.map_ops import PandasMapOpsMixin
import pyspark.sql.streaming
